# L3: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, API and LLM

In [2]:
from crewai import Agent, Task, Crew
from crewai import LLM
from dotenv import load_dotenv
import os

load_dotenv()  # this will load env keys from .env file

True

In [4]:
#api_key = os.getenv('GROQ_API_KEY')
llm = LLM(
    model="ollama/llama3.2",
    #model='groq/gemma2-9b-it',
    temperature = 0.7
)

## Role Playing, Focus and Cooperation

In [8]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True,
	llm=llm
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [ ]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://jadooai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
	llm=llm
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [10]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [11]:
docs_scrape_tool = ScrapeWebsiteTool(
    #website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/",
    website_url="https://jadooai.com/",
    )

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [12]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important task:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [13]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

In [14]:
#pip install langchain-embeddings
embedder={
        "provider": "ollama",
        "config": {"model": "nomic-embed-text"},
 }

### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [15]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=False,
  memory=True,
  llm=llm,
  embedder=embedder
)

/home/tjamil/Desktop/AIS-NLP10S/session-09/.venv/lib/python3.12/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [16]:
inputs = {
    "customer": "TensorDot Solution",
    "person": "Tariq Jamil",
    "inquiry": "I need help with arranging a training course "
               "on NLP during next month. Which subjects would be covered?"
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

2025-04-22 23:39:18,945 - 140322497778112 - rag_storage.py-rag_storage:138 - ERROR: Error during short_term search: Expected Embedings to be non-empty list or numpy array, got [] in query.
2025-04-22 23:39:18,954 - 140322497778112 - rag_storage.py-rag_storage:138 - ERROR: Error during entities search: Expected Embedings to be non-empty list or numpy array, got [] in query.


# Agent: Senior Support Representative
## Task: TensorDot Solution just reached out with a super important task:
I need help with arranging a training course on NLP during next month. Which subjects would be covered?Can you provide guidance?

Tariq Jamil from TensorDot Solution is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.
 

Action '{ "tool": "Read website content", "url": "https://crewai.com" }' don't exist, these are the only available Actions:
Tool Name: Read website content
Tool Arguments: {}
Tool Description: A tool that can be used to read https://jadooai.com/'s content.



# Agent: Senior Support Representative
## Thought: Action: { "tool": "Read website content", "url": "https://crewai.com" }
## Using tool: { "tool": "Read website content", "url": "https://crewai.com" }
## Tool Input: 
"{}"
## Tool Output: 
I encountered an error: Acti

2025-04-22 23:43:33,999 - 140322497778112 - rag_storage.py-rag_storage:109 - ERROR: Error during short_term save: Expected Embedings to be non-empty list or numpy array, got [] in add.




# Agent: Senior Support Representative
## Final Answer: 
This is a comprehensive curriculum for learning Natural Language Processing (NLP) with Python, covering various topics from text preprocessing to large language models and deployment. Here's an overview of the course structure:

**Course Overview**

The 10-week course covers the fundamentals of NLP, including text preprocessing, deep learning, and large language models. It also introduces students to AI agents, chatbots, and tools like LangChain for integrating LLMs into applications.

**Week-by-Week Breakdown**

* **Week 1:** Introduction to NLP, Python basics, and libraries (NLTK, spaCy, Pandas)
* **Week 2:** Text preprocessing (tokenization, stopword removal, stemming)
* **Week 3:** Text representation (bag-of-words, TF-IDF, word embeddings)
* **Week 4:** Deep learning and transformer architecture
* **Week 5:** Advanced NLP with transformers (sentiment analysis, text summarization, translation)
* **Week 6:** Large language m

/home/tjamil/Desktop/AIS-NLP10S/session-09/.venv/lib/python3.12/site-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)


KeyboardInterrupt: 

- Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result.raw)